In [ ]:
import os, sys
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import datetime
import pickle
from lightgbm import *
from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.base import clone
import scipy
from sklearn.metrics import *
from catboost import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.linear_model import *
from xgboost import *
from sklearn.tree import *
from sklearn.neural_network import *

In [ ]:
# X1, y1 ----> 45 features data
# X2, y2 -----> dummy of 45 features
# X3, y3 -----> 74 features data
# X4, y4 -----> new member based 226 features
# x5, y5 -----> new cat features without any label encoding

In [ ]:
[X1, y1, XTest1, testIds1, featureColumns1, groups1] = pickle.load(open('freshPickles/cleanExperimentDataF=45.pk', 'rb'))

In [ ]:
[X2, y2, XTest2, testIds2, featureColumns2, groups2] = pickle.load(open('freshPickles/dummyCleanExperimentData.pk', 'rb'))

In [ ]:
[X3, y3, XTest3, testIds3, featureColumns3, groups3] = pickle.load(open('freshPickles/cleanExperimentDataF=74.pk', 'rb'))

In [ ]:
[X4, y4, XTest4, testIds4, featureColumns4, groups4] = pickle.load(open('freshPickles/cleanExperimentDataF=226.pk', 'rb'))

In [ ]:
[trainDF5, testDF5, y5, testIds5, groups5] = pickle.load(open('freshPickles/catFeaturesdtypecat.pk', 'rb'))

In [ ]:
gfCV5 = list(GroupKFold(n_splits=5).split(X1, y1, groups1))
gfCV10 = list(GroupKFold(n_splits=10).split(X1, y1, groups1))
gfCV20 = list(GroupKFold(n_splits=20).split(X1, y1, groups1))

In [ ]:
def getCrossValPredict(baseModel, X, y, XTest, cv):
    trainCVP = np.zeros((X.shape[0], ))
    testCVP = np.zeros((XTest.shape[0], ))
    i=0
    for ti, vi in cv:
        print("fold "+str(i)+'...')
        model = clone(baseModel)
        model.fit(X[ti], y[ti])
        trainCVP[vi] = model.predict(X[vi])
        testCVP += model.predict(XTest)
        i+=1
    testCVP/=5
    return trainCVP, testCVP

In [ ]:
def getCrossValPredictForDF(baseModel, X, y, XTest, cv):
    trainCVP = np.zeros((X.shape[0], ))
    testCVP = np.zeros((XTest.shape[0], ))
    i=0
    for ti, vi in cv:
        print("fold "+str(i)+'...')
        model = clone(baseModel)
        model.fit(X.iloc[ti], y[ti])
        trainCVP[vi] = model.predict(X.iloc[vi])
        testCVP += model.predict(XTest)
        i+=1
    testCVP/=5
    return trainCVP, testCVP

In [ ]:
pp1 = {'num_leaves': 48, 'subsample_freq': 8, 'colsample_bytree': 0.52, 'max_depth': 16, 'learning_rate': 0.05, 'min_child_weight': 0.001, 'min_split_gain': 0.05, 'n_estimators': 497, 'subsample': 0.92, 'random_state': 7, 'min_child_samples': 20}
pp2 = {'num_leaves': 195, 'subsample_freq': 1, 'colsample_bytree': 0.69, 'max_depth': 7, 'learning_rate': 0.05, 'min_child_weight': 0.001, 'min_split_gain': 0.03, 'n_estimators': 325, 'subsample': 0.61, 'random_state': 7, 'min_child_samples': 50}
pp3 = {'rsm': 0.72, 'model_size_reg': 0.1, 'learning_rate': 0.05, 'leaf_estimation_method': 'Newton', 'depth': 9, 'random_state': 145, 'iterations': 936, 'l2_leaf_reg': 3, 'od_wait': 15, 'one_hot_max_size': 2, 'max_ctr_complexity': 5, 'bagging_temperature': 1}
pp5 = {'num_leaves': 43, 'subsample_freq': 5, 'colsample_bytree': 0.6, 'max_depth': 15, 'learning_rate': 0.07, 'min_child_weight': 0.003, 'min_split_gain': 0, 'n_estimators': 257, 'subsample': 0.68, 'random_state': 7, 'min_child_samples': 40}
pp6 = {'rsm': 0.94, 'model_size_reg': 0.0, 'learning_rate': 0.03, 'leaf_estimation_method': 'Newton', 'depth': 11, 'random_state': 523, 'iterations': 558, 'l2_leaf_reg': 2, 'od_wait': 17, 'one_hot_max_size': 5, 'max_ctr_complexity': 5, 'bagging_temperature': 1.6}  
pp8 = {'num_leaves': 76, 'subsample_freq': 3, 'colsample_bytree': 0.65, 'max_depth': 5, 'learning_rate': 0.05, 'min_child_weight': 0.001, 'min_split_gain': 0.01, 'n_estimators': 273, 'subsample': 0.75, 'random_state': 2019, 'min_child_samples': 20}

model1 = LGBMRegressor(n_jobs=-1, **pp1) #lgbm for 45 f
model2 = LGBMRegressor(n_jobs=-1, **pp2) #lgbm for dummy 45 f
model3 = CatBoostRegressor(silent=True, **pp3) #catboost for 45 f
model4 = LGBMRegressor(n_jobs=-1, random_state=7) #lgbm for new 226 f
model5 = LGBMRegressor(n_jobs=-1, **pp5) #lgbm for 74 f
model6 = CatBoostRegressor(**pp6) #catboost for 74 f
# model7 = CatBoostRegressor(cat_features = **pp6) #catboost for 74 f with cat_features
model8 = LGBMRegressor(n_jobs=-1, **pp8)
model9 = MLPRegressor(hidden_layer_sizes=(32, 8), random_state=7)

In [ ]:
trCV1, teCV1 = getCrossValPredict(model1, X1, y1, XTest1, cv=gfCV5) #lgbm for 45 f

In [ ]:
trCV2, teCV2 = getCrossValPredict(model2, X2, y2, XTest2, cv=gfCV5) #lgbm for dummy 45 f

In [ ]:
trCV3, teCV3 = getCrossValPredict(model3, X1, y1, XTest1, cv=gfCV5) #catboost for 45 f

In [ ]:
trCV4, teCV4 = getCrossValPredict(model4, X4, y4, XTest4, cv=gfCV5) #catboost for 45 f

In [ ]:
trCV5, teCV5 = getCrossValPredict(model5, X3, y3, XTest3, cv=gfCV5) #lgbm for 74 f

In [ ]:
trCV6, teCV6 = getCrossValPredict(model6, X3, y3, XTest3, cv=gfCV5) #catboost for 74 f

In [ ]:
trCV8, teCV8 = getCrossValPredictForDF(model8, trainDF5, y5, testDF5, cv=gfCV5) #lgbm fr

In [ ]:
trCV9, teCV9 = getCrossValPredict(model9, X3, y3, XTest3, cv=gfCV5) #lgbm fr

In [ ]:
l2XTrain = np.hstack((trCV1.reshape((-1, 1)), trCV2.reshape((-1, 1)), trCV3.reshape((-1, 1)), trCV4.reshape((-1, 1)), trCV5.reshape((-1, 1)), trCV6.reshape((-1, 1)), trCV8.reshape((-1, 1))))
l2XTest = np.hstack((teCV1.reshape((-1, 1)), teCV2.reshape((-1, 1)), teCV3.reshape((-1, 1)), teCV4.reshape((-1, 1)), teCV5.reshape((-1, 1)), teCV6.reshape((-1, 1)), teCV8.reshape((-1, 1))))

In [ ]:
l2Models = [
    [RandomForestRegressor(n_jobs=-1, random_state=7), X1, y1, XTest1],
    [ExtraTreesRegressor(n_jobs=-1, random_state=7), X1, y1, XTest1],
    [sgd, X1, y1, XTest1],
    [XGBRegressor(n_jobs=-1, random_state=7), X1, y1, XTest1],
    [AdaBoostRegressor(random_state=7), X1, y1, XTest1],
    [GradientBoostingRegressor(random_state=7), X1, y1, XTest1],
    [DecisionTreeRegressor(random_state=7), X1, y1, XTest1],
    
]

In [ ]:
i=0
pickle.dump([l2XTrain, l2XTest], open('freshPickles/'+'l2ModelData'+str(i)+'.pk', 'wb'))
for model, X, y, XTest in l2Models:
    i+=1
    print(model)
    trCV, teCV = getCrossValPredict(model, X, y, XTest, cv=gfCV5)
    l2XTrain = np.hstack((l2XTrain, trCV.reshape((-1, 1))))
    l2XTest = np.hstack((l2XTest, teCV.reshape((-1, 1))))
    

In [ ]:
l2Models = [
    [RandomForestRegressor(n_jobs=-1, random_state=7), X2, y2, XTest2],
    [ExtraTreesRegressor(n_jobs=-1, random_state=7), X2, y2, XTest2],
    [sgd, X2, y2, XTest2],
    [XGBRegressor(n_jobs=-1, random_state=7), X2, y2, XTest2],
    [AdaBoostRegressor(random_state=7), X2, y2, XTest2],
    [GradientBoostingRegressor(random_state=7), X2, y2, XTest2],
    [DecisionTreeRegressor(random_state=7), X2, y2, XTest2],
    
]

In [ ]:
for model, X, y, XTest in l2Models:
    i+=1
    print(model)
    trCV, teCV = getCrossValPredict(model, X, y, XTest, cv=gfCV5)
    l2XTrain = np.hstack((l2XTrain, trCV.reshape((-1, 1))))
    l2XTest = np.hstack((l2XTest, teCV.reshape((-1, 1))))
    pickle.dump([l2XTrain, l2XTest], open('freshPickles/'+'l2ModelData'+str(i)+'.pk', 'wb'))

In [ ]:
class MyRandomSearch:
    def __init__(self, niter, cv, scoring, model, fileName, params):
        self.niter=niter;
        self.model = model
        self.cv=cv
        self.scoring = scoring
        self.logger = open(fileName, 'a')
        self.params = params
        
    def fit(self, X, y):
        for z in range(self.niter):
            pp = {}
            for key in self.params:
                ri = np.random.randint(0, len(params[key]))
                pp[key] = params[key][ri]
            print(pp)
            model = self.model(**pp)
            scores = cross_val_score(model, X, y, scoring=self.scoring, cv=self.cv, verbose=1, n_jobs=-1)
            meanscore = np.mean(np.sqrt(-1*scores))
            print(str(pp)+" "+str(meanscore))
            self.logger.write(str(pp)+" "+str(meanscore)+"\n")
            self.logger.flush()
        
        

In [ ]:
params = {
    'num_leaves' : list(range(31, 201)),
    'max_depth' : [-1,]+list(range(3, 21)),
    'learning_rate': [0.05, 0.07, 0.09, 0.1],
    'n_estimators': list(range(100, 500)), 
    'subsample': [float(z)/100 for z in list(range(50, 100))],
    'subsample_freq': list(range(0, 10)),
    'colsample_bytree':[float(z)/100 for z in list(range(50, 100))],
    'min_split_gain': [0, 0.01, 0.02, 0.03, 0.04, 0.05],
    'min_child_weight': [0.001, 0.002, 0.003, 0.004, 0.005],
    'min_child_samples': [20, 10, 5, 30, 40, 50],
    'random_state': [7, 42, 2019, 2020],  
    'n_jobs': [-1]
}

model = LGBMRegressor
rs = MyRandomSearch(1000, gfCV5, 'neg_mean_squared_error', model, 'rsp/l2LGBM_N=19.pk', params)
rs.fit(l2XTrain, y1)

In [ ]:
params = {
    'max_depth' : [3, 4, 5],
    'learning_rate': [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1],
    'n_estimators': list(range(50, 150)),
    'n_jobs': [-1],
    'gamma': [0, 0.001, 0.002, 0.003, 0.004],
    'max_delta_step': [1,],
 
}


model = XGBRegressor
rs = MyRandomSearch(1000, gfCV5, 'neg_mean_squared_error', model, 'rsp/l2XGB_N=19.pk', params)
rs.fit(l2XTrain, y1)

In [ ]:
params = {
    'n_estimators': list(range(10, 200)),
    'max_depth': list(range(5, 20)),
    'min_samples_split': list(range(2, 10)),
    'min_samples_leaf': list(range(1, 10)),
    'max_features': ['sqrt', 'log2']+[float(z)/100 for z in list(range(70, 100))],
    'bootstrap': [True, False],
    'random_state': list(range(7, 2020)),
                      
}

model = ExtraTreesRegressor
rs = MyRandomSearch(1000, gfCV5, 'neg_mean_squared_error', model, 'rsp/l2ET_N=19.pk', params)
rs.fit(l2XTrain, y1)

In [ ]:
params = {
    'penalty': ['l2', 'l1', 'elasticnet'],
    'random_state': list(range(7, 2020)),
    'alpha': [1e-4, 1e-4, 1e-3],
    'l1_ratio': [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.05],
    'max_iter': list(range(5, 1000)),
    'tol': [1e-5, 1e-4, 1e-3],
    'average': [True, True, False],
    
}

model = SGDRegressor

scaler = StandardScaler()
l2XTrainSc = scaler.fit_transform(l2XTrain)
l2XTestSc = scaler.transform(l2XTest)

rs = MyRandomSearch(1000, gfCV5, 'neg_mean_squared_error', model, 'rsp/l2ET_N=19.pk', params)
rs.fit(l2XTrainSc, y1)

In [ ]:
pplgb = {'num_leaves': 114, 'subsample_freq': 4, 'colsample_bytree': 0.54, 'max_depth': 3, 'learning_rate': 0.07, 'min_child_weight': 0.004, 'min_split_gain': 0.05, 'n_estimators': 295, 'subsample': 0.61, 'random_state': 7, 'min_child_samples': 10}
ppet = {'bootstrap': False, 'min_samples_leaf': 9, 'n_estimators': 129, 'min_samples_split': 7, 'random_state': 1458, 'max_features': 0.74, 'max_depth': 16}
ppsgd = {'l1_ratio': 0.2, 'average': False, 'max_iter': 981, 'penalty': 'l1', 'random_state': 1387, 'tol': 0.001, 'alpha': 0.001}


finalModels = [
    LGBMRegressor(n_jobs=-1, **pplgb),
    XGBRegressor(n_jobs=-1, random_state=7),
    Pipeline([
        ('scaler', StandardScaler()),
        ('sgd', SGDRegressor(**ppsgd)),
    ]),
]


yPred = np.zeros((len(testDF5),))

for model in finalModels:
    model.fit(l2XTrain, y1)
    yPred += model.predict(l2XTest)

In [ ]:
yPred = yPred/3

In [ ]:
submissionDF = pd.read_csv('dataset/sample_submission.csv')

In [ ]:
submissionDF['reservation_id'] = testIds1
submissionDF['amount_spent_per_room_night_scaled'] = yPred
submissionDF.to_csv('submissions/submission.csv', index=False)